# Tratamento de dados

### passo 1 - importar bibliotecas 

In [1]:
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib.colors import ListedColormap, BoundaryNorm
import numpy as np
import pandas as pd
from scipy import signal, ndimage

### passo 2 - importar csv de referência
    - imprtar utilizando pandas
    - imprimir gráficos de cada canal

In [9]:
datasheet = pd.read_csv("../dataseets/original.csv", delimiter=";")

colors = {"hand_open":"orange", "hand_flex_curl":"blue"}

# laco para gerar os gráficos baseados na label (hand-open ou hand-flex-curl)

for i in range(0,4):
    plt.figure(figsize=(20,6))
    plt.scatter(range(len(datasheet[f"ch{i}"])), datasheet[f"ch{i}"], color=datasheet["gesture"].map(colors), s=1)
    plt.title("Original signal colored by gesture")
    plt.suptitle(f"Channel {i}")
    plt.savefig(f"../graphs/original/original-labeled-ch{i}.png")
    plt.clf()

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

### passo 3 - Aplicar filtro da mediana
    - reduzir o ruído de impulso
    - salvar em novo csv
    - gerar gráficos para cada canal

In [11]:
# copiando datasheet
median = datasheet.copy()

# laco para aplicar o filtro da mediana nos 4 canais
for i in range(0,4):
    # aplica o filtro da mediana no canal i
    median[f"ch{i}"] = signal.medfilt(median[f"ch{i}"], 9)
    # gera gráficos para o canal i
    plt.figure(figsize=(20,6))
    plt.scatter(range(len(median[f"ch{i}"])), median[f"ch{i}"], color=median["gesture"].map(colors), s=1)
    plt.title("Filtered by median signal colored by gesture")
    plt.suptitle(f"Channel {i}")
    plt.savefig(f"../graphs/median/median-labeled-ch{i}.png")
    plt.clf()
    
# salvando dados em um novo arquivo .csv
median.to_csv("../datasheets/median.csv")

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

### passo 4 -Aplicar média móvel
    - suavizar sinal
    - salvar em um novo csv
    - gerar gráficos para cada canal

In [12]:
# copiando datasheet
mov_avg = median.copy()

# laco para aplicar o filtro da mediana nos 4 canais
for i in range(0,4):
    # aplica o media movel no canal i
    mov_avg['ch0'] = median.ch0.rolling(100).mean()
    mov_avg['ch1'] = median.ch1.rolling(100).mean()
    mov_avg['ch2'] = median.ch2.rolling(100).mean()
    mov_avg['ch3'] = median.ch3.rolling(100).mean()
    
    # gera gráficos para o canal i
    plt.figure(figsize=(20,6))
    plt.scatter(range(len(mov_avg[f"ch{i}"])), mov_avg[f"ch{i}"], color=mov_avg["gesture"].map(colors), s=1)
    plt.title("rolling mean applied to the signal colored by gesture")
    plt.suptitle(f"Channel {i}")
    plt.savefig(f"../graphs/mov_avg/mov_avg-labeled-ch{i}.png")
    plt.clf()
    
# salvando dados em um novo arquivo .csv
mov_avg.to_csv("../datasheets/mov_avg.csv")

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

### passo 5 - encontrar o zero do sinal
    - tecnica utilizada: subtrair o valor de um intervalo por sua media móvel
    - salvar em um novo csv
    - gerar graficos para cada canal

In [17]:
# copiando datasheet
centered = mov_avg.copy()

# laco para aplicar o filtro da mediana nos 4 canais
for i in range(0,4):
    # aplica a poeracao de centralização no canal i
    for j in range (0, len(centered[f"ch{i}"]), 100):
        centered[f"ch{i}"][j:j+100] = centered[f"ch{i}"][j:j+100] - centered[f"ch{i}"][j:j+100].mean()
    # gera gráficos para o canal i
    plt.figure(figsize=(20,6))
    plt.scatter(range(len(centered[f"ch{i}"])), centered[f"ch{i}"], color=centered["gesture"].map(colors), s=1)
    plt.plot(range(len(centered[f"ch{i}"])), np.zeros_like(centered[f"ch{i}"]), color="black")
    plt.title("signal centered on zero colored by gesture")
    plt.suptitle(f"Channel {i}")
    plt.savefig(f"../graphs/centered/centered-labeled-ch{i}.png")
    plt.clf()
    
# salvando dados em um novo arquivo .csv
centered.to_csv("../datasheets/centered.csv")

/tmp/ipykernel_4312/3034585980.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  centered[f"ch{i}"][j:j+100] = centered[f"ch{i}"][j:j+100] - centered[f"ch{i}"][j:j+100].mean()
/tmp/ipykernel_4312/3034585980.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  centered[f"ch{i}"][j:j+100] = centered[f"ch{i}"][j:j+100] - centered[f"ch{i}"][j:j+100].mean()
/tmp/ipykernel_4312/3034585980.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>

<Figure size 2000x600 with 0 Axes>